In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import cartopy.feature as cfeature
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import matplotlib.pyplot as plt
import time
import importlib
import stormcoupling as sc
import glob
from scipy import stats
import scipy
from geopy import distance
from scipy import interpolate
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import cm
import xlrd
import csv
import os
import math
import matplotlib.colors
import metpy
from metpy.interpolate import interpolate_to_isosurface
from metpy.calc import dewpoint_from_specific_humidity
from metpy.calc import equivalent_potential_temperature
from matplotlib.ticker import LogFormatterSciNotation
import matplotlib.patheffects as pe
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from metpy.units import units 
import metpy.calc as mpcalc
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import imageio.v2 as imageio
from PIL import Image, ImageOps
importlib.reload(sc)

<module 'stormcoupling' from '/lustre06/project/6084782/shared/gabe-fall-2025/TEtracking/stormcoupling.py'>

In [2]:
#### takes output data from StitchNodes (tracks) and organizes it into datafram with individual track IDs

def parse_te_tracks(filename):
    tracks = []
    trackid = -1
    with open(filename) as f:
        current_id = None
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            if parts[0] == "start":
                length = int(parts[1])  # track ID
                trackid = trackid+1
            else:
                lon, lat, msl, year, month, day, hour = parts[-7:]
                tracks.append({
                    "id": trackid,
                    "length": length,
                    "lon": float(lon),
                    "lat": float(lat),
                    "msl": float(msl),
                    "time": pd.Timestamp(int(year), int(month), int(day), int(hour))
                })
    return pd.DataFrame(tracks)

In [3]:
#### locates points from track dataframe at specific time

def findDataAtTime(year,month,day,hour,var,tracks,data):
    timestamp = f"{year}-{month}-{day}T{hour}:00"
    indices_time1 = tracks[tracks['time'] == timestamp]
    data_time1 = data[var].sel(valid_time=timestamp)
    return indices_time1, data_time1, timestamp

In [4]:
#### locates points from track dataframe at specific time (returns both cyclones (data1) and anticyclones (data2))


def findDataAtTime2(year,month,day,hour,var,tracks1,tracks2,data):
    timestamp = f"{year}-{month}-{day}T{hour}:00"
    indices_time1 = tracks1[tracks1['time'] == timestamp]
    indices_time2 = tracks2[tracks2['time'] == timestamp]
    data_time1 = data[var].sel(valid_time=timestamp)
    return indices_time1, indices_time2, data_time1, timestamp

In [5]:
#### locates points from track dataframe at specific time (returns both cyclones (data1) and anticyclones (data2)) but takes timestamp object


def findDataAtTime2_timestamp(timestamp,var,tracks1,tracks2,data):
    indices_time1 = tracks1[tracks1['time'] == timestamp]
    indices_time2 = tracks2[tracks2['time'] == timestamp]
    data_time1 = data[var].sel(valid_time=timestamp)
    return indices_time1, indices_time2, data_time1, timestamp

In [6]:
## plots contour map of MSL data (takes output of findDataAtTime functions)

def plotMSLAtTime2(output_FindData, bounds=[-175, -30, 20, 75]):
    indices1 = output_FindData[0]
    indices2 = output_FindData[1]
    data1 = output_FindData[2]
    fig=plt.figure(figsize=(12,8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    #ax.stock_img()
    ax.coastlines(color='gray')
    ax.gridlines(draw_labels=True)
    ax.add_feature(
        cfeature.LAND, 
        facecolor='palegoldenrod',   # any Matplotlib color
        zorder=0                 
    )

    ax.add_feature(
        cfeature.OCEAN, 
        facecolor='lightblue',
        zorder=0
    )
    ax.add_feature(
        cfeature.LAKES, 
        facecolor='lightblue',
        zorder=0
    )
    ax.add_feature(
        cfeature.BORDERS, 
        facecolor='gray',
        zorder=0
    )
    ax.add_feature(
        cfeature.RIVERS, 
        facecolor='lightblue',
        zorder=0
    )
    # basic stats
    msl_hpa = (data1 / 100.).values
    lon = data1['longitude'].values
    lat = data1['latitude'].values
    vmin = float(np.nanmin(msl_hpa))
    vmax = float(np.nanmax(msl_hpa))

    
    coarse_step = 4.0    # coarse contour spacing (hPa)
    fine_step   = 4.0    # fine contour spacing near minima (hPa)
    fine_range  = 12.0   # create finer spacing for values within (vmin, vmin+fine_range)

    # make coarse levels (descending or ascending doesn't matter for list)
    coarse_levels = np.arange(np.floor(vmin - 0.5*coarse_step),
                              np.ceil(vmax + 0.5*coarse_step),
                              coarse_step)

    # make fine levels near minima
    fine_levels = np.arange(np.floor(vmin - 0.5*fine_step),
                            vmin + fine_range + 0.0001,
                            fine_step)

    # combine, remove duplicates and sort
    levels = np.unique(np.concatenate([coarse_levels, fine_levels]))
    levels = np.sort(levels)
    # Plot center of each track at time t
    for index in zip(indices1.lon, indices1.lat, indices1.msl):
        mslval = round(index[2]/100)
        ax[0].text(index[0], index[1], 'L', c='red', fontsize=15, ha='center', va='center', fontweight='bold', zorder=4)
        ax[0].text(index[0], index[1]-1.2, mslval, c='red', fontsize=10, ha='center', va='top', fontweight='bold', zorder=4)
    for index in zip(indices2.lon, indices2.lat, indices2.msl):
        mslval = round(index[2]/100)
        ax[0].text(index[0], index[1], 'H', c='blue', fontsize=15, ha='center', va='center', fontweight='bold', zorder=4)
        ax[0].text(index[0], index[1]-1.2, mslval, c='blue', fontsize=10, ha='center', va='top', fontweight='bold', zorder=4)
    contour = ax.contour(
        data1['longitude'],
        data1['latitude'],
        data1 / 100,  # convert from Pa to hPa 
        levels=levels,
        transform=ccrs.PlateCarree(),
        colors='black',
        linewidths=1,
        zorder=1
    )

    ax.clabel(contour, inline=True, fontsize=8, fmt="%.0f",zorder=3)
    ax.set_extent(bounds, crs=ccrs.PlateCarree())
    ax.set_title(f'msl, time = {output_FindData[3]}')
    #plt.savefig('test.jpg', bbox_inches='tight')
    plt.savefig(f'plots/random/msl_{output_FindData[3]}.png', bbox_inches='tight')
    plt.show()

In [7]:
## same as previous but gradient plot for meteorological variables

def plotGRADAtTime2(output_FindData, cmap1, vmin1=None, vmax1=None):
    indices1 = output_FindData[0]
    indices2 = output_FindData[1]
    data1 = output_FindData[1]
    p = None
    if 'pressure_level' in list(data1.coords):
        first_p = data1.pressure_level.data[0]
        print(f'Pressure level data found, defaulting to {first_p}')
        data1 = data1.sel(pressure_level=first_p)
        p = first_p
    var = data1.name
    fig=plt.figure(figsize=(12,8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(color='black')
    ax.gridlines(draw_labels=True)
    ax.add_feature(
        cfeature.LAND, 
        facecolor='palegoldenrod',
        zorder=0                 # draw behind contours
    )

    ax.add_feature(
        cfeature.OCEAN, 
        facecolor='lightblue',
        zorder=0
    )
    # Plot each cyclone track
    for index in zip(indices1.lon, indices1.lat, indices1.msl):
        mslval = round(index[2]/100)
        ax[0].text(index[0], index[1], 'L', c='red', fontsize=15, ha='center', va='center', fontweight='bold', zorder=4)
        ax[0].text(index[0], index[1]-1.2, mslval, c='red', fontsize=10, ha='center', va='top', fontweight='bold', zorder=4)
    for index in zip(indices2.lon, indices2.lat, indices2.msl):
        mslval = round(index[2]/100)
        ax[0].text(index[0], index[1], 'H', c='blue', fontsize=15, ha='center', va='center', fontweight='bold', zorder=4)
        ax[0].text(index[0], index[1]-1.2, mslval, c='blue', fontsize=10, ha='center', va='top', fontweight='bold', zorder=4)
    im = ax.pcolormesh(
        data1['longitude'],
        data1['latitude'],
        data1,  
        transform=ccrs.PlateCarree(),
        cmap=cmap1,
        linewidths=1,
        zorder=1,
        vmin=vmin1,
        vmax=vmax1
    )
    cbar = plt.colorbar(im, orientation='horizontal', pad=0.05, aspect=50)
    cbar.set_label(f'{var}, {data1.units}')
    #ax.clabel(im, inline=True, fontsize=8, fmt="%.0f")
    ax.set_extent([-175, -30, 20, 75], crs=ccrs.PlateCarree())
    ax.set_title(f'{var}, time = {output_FindData[2]}')
    #plt.savefig('test.jpg', bbox_inches='tight')
    plt.savefig(f'plots/random/{var}{p}_{output_FindData[2]}.png', bbox_inches='tight')
    plt.show()

In [8]:
## plots two panel contour and gradient map of MSL & variable of choice data data (takes output of findDataAtTime functions for both msl and the variable)

def plotMSLAndGradAtTime_twopanel(output_FindData, output_FindData1, cmap1, vmin1=None, vmax1=None, bounds=[-175, -30, 20, 75]):
    indices1 = output_FindData[0]
    indices2 = output_FindData[1]
    data1 = output_FindData[2]
    data2 = output_FindData1[2]
    p = None
    if 'pressure_level' in list(data2.coords):
        first_p = data2.pressure_level.data[0]
        print(f'Pressure level data found, defaulting to {first_p}')
        data2 = data2.sel(pressure_level=first_p)
        p = first_p
    var = data2.name
    fig, axes = plt.subplots(
    nrows=2, ncols=1,
    subplot_kw={'projection': ccrs.PlateCarree()},
    figsize=(18, 14)
    )
    #ax.stock_img()
    axes[0].coastlines(color='gray')
    axes[0].gridlines(draw_labels=True)
    axes[1].coastlines(color='gray')
    axes[1].gridlines(draw_labels=True)
    axes[0].add_feature(
        cfeature.LAND, 
        facecolor='palegoldenrod',   # any Matplotlib color
        zorder=0                 
    )

    axes[0].add_feature(
        cfeature.OCEAN, 
        facecolor='lightblue',
        zorder=0
    )
    axes[0].add_feature(
        cfeature.LAKES, 
        facecolor='lightblue',
        zorder=0
    )
    axes[0].add_feature(
        cfeature.BORDERS, 
        facecolor='gray',
        zorder=0
    )
    axes[1].add_feature(
        cfeature.BORDERS, 
        facecolor='gray',
        zorder=0
    )
    axes[0].add_feature(
        cfeature.RIVERS, 
        facecolor='lightblue',
        zorder=0
    )
    # basic stats
    msl_hpa = (data1 / 100.).values
    lon = data1['longitude'].values
    lat = data1['latitude'].values
    vmin = float(np.nanmin(msl_hpa))
    vmax = float(np.nanmax(msl_hpa))

    
    coarse_step = 4.0    # coarse contour spacing (hPa)
    fine_step   = 4.0    # fine contour spacing near minima (hPa)
    fine_range  = 12.0   # create finer spacing for values within (vmin, vmin+fine_range)

    # make coarse levels (descending or ascending doesn't matter for list)
    coarse_levels = np.arange(np.floor(vmin - 0.5*coarse_step),
                              np.ceil(vmax + 0.5*coarse_step),
                              coarse_step)

    # make fine levels near minima
    fine_levels = np.arange(np.floor(vmin - 0.5*fine_step),
                            vmin + fine_range + 0.0001,
                            fine_step)

    # combine, remove duplicates and sort
    levels = np.unique(np.concatenate([coarse_levels, fine_levels]))
    levels = np.sort(levels)
    # Plot center of each track at time t
    for index in zip(indices1.lon, indices1.lat, indices1.msl):
        mslval = round(index[2]/100)
        axes[0].text(index[0], index[1], 'L', c='red', fontsize=15, ha='center', va='center', fontweight='bold', zorder=4)
        axes[0].text(index[0], index[1]-1.2, mslval, c='red', fontsize=10, ha='center', va='top', fontweight='bold', zorder=4)
    for index in zip(indices2.lon, indices2.lat, indices2.msl):
        mslval = round(index[2]/100)
        axes[0].text(index[0], index[1], 'H', c='blue', fontsize=15, ha='center', va='center', fontweight='bold', zorder=4)
        axes[0].text(index[0], index[1]-1.2, mslval, c='blue', fontsize=10, ha='center', va='top', fontweight='bold', zorder=4)
    for index in zip(indices1.lon, indices1.lat, indices1.msl):
        mslval = round(index[2]/100)
        axes[1].text(index[0], index[1], 'L', c='red', fontsize=15, ha='center', va='center', fontweight='bold', zorder=4)
        axes[1].text(index[0], index[1]-1.2, mslval, c='red', fontsize=10, ha='center', va='top', fontweight='bold', zorder=4)
    for index in zip(indices2.lon, indices2.lat, indices2.msl):
        mslval = round(index[2]/100)
        axes[1].text(index[0], index[1], 'H', c='blue', fontsize=15, ha='center', va='center', fontweight='bold', zorder=4)
        axes[1].text(index[0], index[1]-1.2, mslval, c='blue', fontsize=10, ha='center', va='top', fontweight='bold', zorder=4)
    contour = axes[0].contour(
        data1['longitude'],
        data1['latitude'],
        data1 / 100,  # convert from Pa to hPa 
        levels=levels,
        transform=ccrs.PlateCarree(),
        colors='black',
        linewidths=1,
        zorder=1
    )
    im = axes[1].pcolormesh(
        data2['longitude'],
        data2['latitude'],
        data2,  
        transform=ccrs.PlateCarree(),
        cmap=cmap1,
        linewidths=1,
        zorder=1,
        vmin=vmin1,
        vmax=vmax1
    )
    axes[0].clabel(contour, inline=True, fontsize=8, fmt="%.0f",zorder=3)
    axes[0].set_extent(bounds, crs=ccrs.PlateCarree())
    axes[0].set_title(f'msl, time = {output_FindData[3]}')
    #cbar = fig.colorbar(im, ax=axes[1], orientation='horizontal', pad=0.05, aspect=50, fraction=0.03)
    cb_ax = inset_axes(axes[1], width="100%", height="5%", loc='lower center', borderpad=-3.5)
    cbar = plt.colorbar(im, cax=cb_ax, orientation='horizontal')
    cbar.set_label(f'{var}_{first_p}, {data2.units}')
    #ax.clabel(im, inline=True, fontsize=8, fmt="%.0f")
    axes[1].set_extent(bounds, crs=ccrs.PlateCarree())
    #axes[1].set_title(f'{var}, time = {output_FindData[3]}')
    #plt.savefig('test.jpg', bbox_inches='tight')
    plt.savefig(f'plots/random/msl_{var}_2panel_{output_FindData[3]}.png', bbox_inches='tight')
    plt.show()

In [9]:
start1 = pd.Timestamp("2015-04-02 00:00:00")

In [188]:
end1 = pd.Timestamp("2015-04-05 00:00:00")

In [10]:
## creates and saves png frames using twopanel function starting at start and ending at end in 3 hour increments

def createGifFrames_twopanel(start, end, tracks_c, tracks_ac, data1, data2, var, cmap1, vmin1=None, vmax1=None, bounds=[-175, -30, 20, 75]):
    for time in pd.date_range(start=start, end=end, freq='3h'):
        print(time)
        output1 = findDataAtTime2_timestamp(time,'msl', tracks_c, tracks_ac, data1)
        output2 = findDataAtTime2_timestamp(time, var, tracks_c, tracks_ac, data2)
        plotMSLAndGradAtTime_twopanel(output1, output2, cmap1=cmap1, vmin1=vmin1, vmax1=vmax1, bounds=bounds)

In [11]:
## same as previous but for just msl

def createGifFrames_msl(start, end, tracks_c, tracks_ac, data1, bounds=[-175, -30, 20, 75]):
    for time in pd.date_range(start=start, end=end, freq='3h'):
        print(time)
        output1 = findDataAtTime2_timestamp(time,'msl', tracks_c, tracks_ac, data1)
        plotMSLAtTime2(output1, bounds=bounds)

In [ ]:
createGifFrames_twopanel(start1, end1, msl_200_vo, anticyclone_150, msl_u_v, idk, var='vo', cmap1='PRGn', vmin1=-0.0004, vmax1=0.0004)

In [211]:
## creates gif from frames

folder = "plots/random/frames3"
output_gif = "animation_2.gif"

images = sorted(
    [img for img in os.listdir(folder) if img.endswith((".png", ".jpg"))]
)

frames = []
for img_name in images:
    img_path = os.path.join(folder, img_name)
    frames.append(imageio.imread(img_path))

imageio.mimsave(output_gif, frames, duration=250, loop=0)

print("✅ GIF saved as", output_gif)

✅ GIF saved as animation_2.gif


In [207]:
## helper function to make frames format fit ffmpeg expected style (proper aspect ratio)

def make_even(im):
    w, h = im.size
    new_w = w + (w % 2)
    new_h = h + (h % 2)
    return ImageOps.pad(im, (new_w, new_h))

In [209]:
## make mp4 (instead of gif)


folder = "plots/random/frames3"
output_mp4 = "animation_mp4.mp4"

images = sorted([img for img in os.listdir(folder) if img.endswith(".png")])
frames = []

for img_name in images:
    path = os.path.join(folder, img_name)
    im = Image.open(path)
    im = make_even(im)
    im = im.convert("RGB")       
    frames.append(np.array(im))  

imageio.mimsave(output_mp4, frames, fps=5)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1396, 1232) to (1408, 1232) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x557ff3980e40] Warning: data is not aligned! This can lead to a speed loss


In [ ]:
output_FindData_1 = findDataAtTime2('2015','04','02','15','msl', msl_200_vo, anticyclone_150, msl_u_v)
output_FindData_2 = findDataAtTime2('2015','04','02','15','vo', msl_200_vo, anticyclone_150, idk)


In [ ]:
plotMSLAndGradAtTime_twopanel(output_FindData_1, output_FindData_2, 'PRGn', vmin1=-0.0004, vmax1=0.0004)

In [ ]:
## old msl plotter, deprecated

def plotMSLAtTime(output_FindData, bounds=[-175, -30, 20, 75]):
    indices1 = output_FindData[0]
    data1 = output_FindData[1]
    fig=plt.figure(figsize=(12,8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(color='gray')
    ax.gridlines(draw_labels=True)
    
    ax.add_feature(
        cfeature.LAND, 
        facecolor='palegoldenrod',   # any Matplotlib color
        zorder=0                 
    )

    ax.add_feature(
        cfeature.OCEAN, 
        facecolor='lightblue',
        zorder=0
    )
    
    # basic stats
    msl_hpa = (data1 / 100.).values
    lon = data1['longitude'].values
    lat = data1['latitude'].values
    vmin = float(np.nanmin(msl_hpa))
    vmax = float(np.nanmax(msl_hpa))

    
    coarse_step = 4.0    # coarse contour spacing (hPa)
    fine_step   = 4.0    # fine contour spacing near minima (hPa)
    fine_range  = 12.0   # create finer spacing for values within (vmin, vmin+fine_range)

    # make coarse levels (descending or ascending doesn't matter for list)
    coarse_levels = np.arange(np.floor(vmin - 0.5*coarse_step),
                              np.ceil(vmax + 0.5*coarse_step),
                              coarse_step)

    # make fine levels near minima
    fine_levels = np.arange(np.floor(vmin - 0.5*fine_step),
                            vmin + fine_range + 0.0001,
                            fine_step)

    # combine, remove duplicates and sort
    levels = np.unique(np.concatenate([coarse_levels, fine_levels]))
    levels = np.sort(levels)
    # Plot center of each track at time t
    for index in zip(indices1.lon, indices1.lat):
        ax.text(index[0], index[1], 'L', c='red', fontsize=15, ha='center', va='center', fontweight='bold', zorder=2)
    contour = ax.contour(
        data1['longitude'],
        data1['latitude'],
        data1 / 100,  # convert from Pa to hPa 
        levels=levels,
        transform=ccrs.PlateCarree(),
        colors='black',
        linewidths=1,
        zorder=1
    )

    ax.clabel(contour, inline=True, fontsize=8, fmt="%.0f")
    ax.set_extent(bounds, crs=ccrs.PlateCarree())
    ax.set_title(f'msl, time = {output_FindData[2]}')
    #plt.savefig('test.jpg', bbox_inches='tight')
    plt.savefig(f'plots/random/msl_{output_FindData[2]}.png', bbox_inches='tight')
    plt.show()

In [ ]:
## old grad plotter, deprecated

def plotGRADAtTime(output_FindData, cmap1, vmin1=None, vmax1=None):
    indices1 = output_FindData[0]
    data1 = output_FindData[1]
    p = None
    if 'pressure_level' in list(data1.coords):
        first_p = data1.pressure_level.data[0]
        print(f'Pressure level data found, defaulting to {first_p}')
        data1 = data1.sel(pressure_level=first_p)
        p = first_p
    var = data1.name
    fig=plt.figure(figsize=(12,8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(color='black')
    ax.gridlines(draw_labels=True)
    ax.add_feature(
        cfeature.LAND, 
        facecolor='palegoldenrod',
        zorder=0                 # draw behind contours
    )

    ax.add_feature(
        cfeature.OCEAN, 
        facecolor='lightblue',
        zorder=0
    )
    # Plot each cyclone track
    for index in zip(indices1.lon, indices1.lat):
        ax.scatter(index[0], index[1], c='red', zorder=2)
    im = ax.pcolormesh(
        data1['longitude'],
        data1['latitude'],
        data1,  
        transform=ccrs.PlateCarree(),
        cmap=cmap1,
        linewidths=1,
        zorder=1,
        vmin=vmin1,
        vmax=vmax1
    )
    cbar = plt.colorbar(im, orientation='horizontal', pad=0.05, aspect=50)
    cbar.set_label(f'{var}, {data1.units}')
    #ax.clabel(im, inline=True, fontsize=8, fmt="%.0f")
    ax.set_extent([-175, -30, 20, 75], crs=ccrs.PlateCarree())
    ax.set_title(f'{var}, time = {output_FindData[2]}')
    #plt.savefig('test.jpg', bbox_inches='tight')
    plt.savefig(f'plots/random/{var}{p}_{output_FindData[2]}.png', bbox_inches='tight')
    plt.show()

In [ ]:
## ## old msl and grad plotter, deprecated

def plotMSLAndGradAtTime(output_FindData1, output_FindData2, cmap1, vmin1=None, vmax1=None, bounds=[-175, -30, 20, 75]):
    indices1 = output_FindData1[0]
    data1 = output_FindData1[1]
    data2 = output_FindData2[1]
    p = None
    if 'pressure_level' in list(data2.coords):
        first_p = data2.pressure_level.data[0]
        print(f'Pressure level data found, defaulting to {first_p}')
        data2 = data2.sel(pressure_level=first_p)
        p = first_p
    #return data2
    var = data2.name
    fig=plt.figure(figsize=(12,8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(color='black', linewidth=0.5)
    ax.gridlines(draw_labels=True)
    ax.add_feature(
        cfeature.LAND, 
        facecolor='palegoldenrod',   # any Matplotlib color
        zorder=0                 # draw behind contours
    )

    ax.add_feature(
        cfeature.OCEAN, 
        facecolor='lightblue',
        zorder=0
    )
    # basic stats
    msl_hpa = (data1 / 100.).values
    lon = data1['longitude'].values
    lat = data1['latitude'].values
    vmin = float(np.nanmin(msl_hpa))
    vmax = float(np.nanmax(msl_hpa))

    
    coarse_step = 4.0    # coarse contour spacing (hPa)
    fine_step   = 4.0    # fine contour spacing near minima (hPa)
    fine_range  = 12.0   # create finer spacing for values within (vmin, vmin+fine_range)

    # make coarse levels (descending or ascending doesn't matter for list)
    coarse_levels = np.arange(np.floor(vmin - 0.5*coarse_step),
                              np.ceil(vmax + 0.5*coarse_step),
                              coarse_step)

    # make fine levels near minima
    fine_levels = np.arange(np.floor(vmin - 0.5*fine_step),
                            vmin + fine_range + 0.0001,
                            fine_step)

    # combine, remove duplicates and sort
    levels = np.unique(np.concatenate([coarse_levels, fine_levels]))
    levels = np.sort(levels)
    # Plot each cyclone track
    for index in zip(indices1.lon, indices1.lat):
        ax.scatter(index[0], index[1], c='red', zorder=2)
    im = ax.pcolormesh(
        data2['longitude'],
        data2['latitude'],
        data2,  
        transform=ccrs.PlateCarree(),
        cmap=cmap1,
        linewidths=1,
        zorder=1,
        vmin=vmin1,
        vmax=vmax1
    )
    cbar = plt.colorbar(im, orientation='horizontal', pad=0.05, aspect=50)
    cbar.set_label(f'{var}, {data2.units}')
    contour = ax.contour(
        data1['longitude'],
        data1['latitude'],
        data1 / 100,  # convert from Pa to hPa
        levels=levels,
        transform=ccrs.PlateCarree(),
        colors='black',
        linewidths=1,
        zorder=1
    )

    ax.clabel(contour, inline=True, fontsize=8, fmt="%.0f")
    ax.set_extent(bounds, crs=ccrs.PlateCarree())
    ax.set_title(f'msl (hPa) and {var}_{first_p}, time = {output_FindData1[2]}')
    #plt.savefig('test.jpg', bbox_inches='tight')
    plt.savefig(f'plots/random2/msl_{var}_{output_FindData1[2]}.png', bbox_inches='tight')
    plt.show()

In [190]:
msl_200_vo = parse_te_tracks("data/TEtextfiles/StitchNodes/cyclone_SN.txt")

In [191]:
anticyclone_150 = parse_te_tracks("data/TEtextfiles/StitchNodes/anticyclone_SN_3.txt")

In [192]:
msl_u_v = xr.open_dataset('/home/glach/projects/def-rfajber/shared/gabe-fall-2025/TEtracking/data/era5-raw/2015-JFMA-msl-uv.nc')

In [193]:
# [-175, -30, 20, 75]

In [194]:
t_q = xr.open_dataset('/home/glach/projects/def-rfajber/shared/gabe-fall-2025/TEtracking/data/era5-raw/2015-q-t-850.nc')

In [195]:
idk = xr.open_dataset('/home/glach/projects/def-rfajber/shared/gabe-fall-2025/TEtracking/data/era5-raw/2015-v-300.nc')

In [196]:
idk2 = xr.open_dataset('/home/glach/projects/def-rfajber/shared/gabe-fall-2025/TEtracking/data/era5-raw/vort-2015-JFMA.nc')

In [ ]:
findDataAtTime('2015','01','18','06','msl', msl_200_vo, msl_u_v)[1].units

In [ ]:
### general case. working as intended (i think)

In [ ]:
findDataAtTime2('2015','04','02','15','msl', msl_200_vo, anticyclone_150, msl_u_v)[3]

In [ ]:
output_FindData_1 = findDataAtTime2('2015','04','02','15','msl', msl_200_vo, anticyclone_150, msl_u_v)
output_FindData_2 = findDataAtTime2('2015','04','02','15','t', msl_200_vo, anticyclone_150, t_q)


In [ ]:
output_FindData_1[3]

In [ ]:
plotMSLAndGradAtTime_twopanel(output_FindData_1, output_FindData_2, 'coolwarm')

In [ ]:
## cyclone

plotMSLAtTime2(findDataAtTime2('2015','04','02','15','msl', msl_200_vo, anticyclone_150, msl_u_v))

In [ ]:
## cyclone

plotMSLAtTime2(findDataAtTime2('2015','04','02','15','msl', msl_200_vo, anticyclone_150, msl_u_v))

In [ ]:
## anticyclone

plotMSLAtTime(findDataAtTime('2015','01','18','06','msl', anticyclone_150, msl_u_v))

In [ ]:
## cyclone, u10 wind

plotGRADAtTime(findDataAtTime('2015','01','18','06','u10', msl_200_vo, msl_u_v), 'viridis')

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','01','18','06','t', msl_200_vo, t_q), 'coolwarm')

In [ ]:
msl_200_vo[msl_200_vo['msl'] == msl_200_vo.msl.min()]

In [ ]:
## unusual case

plotMSLAtTime(findDataAtTime('2015','04','02','00','msl', msl_200_vo, msl_u_v))

In [ ]:
days = np.array(['02', '03'])
hours = np.array(['00', '03', '06', '09', '12', '15', '18', '21'])

In [ ]:
for day in days: 
    for hour in hours:
        plotMSLAtTime(findDataAtTime('2015','04',day,hour,'msl', msl_200_vo, msl_u_v))

In [ ]:
## unusual case

plotMSLAtTime(findDataAtTime('2015','04','04','00','msl', msl_200_vo, msl_u_v))

In [ ]:
days = np.array(['13', '14'])
hours = np.array(['00', '03', '06', '09', '12', '15', '18', '21'])

In [ ]:
for day in days: 
    for hour in hours:
        plotMSLAtTime(findDataAtTime('2015','03',day,hour,'msl', anticyclone_150, msl_u_v))

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','04','02','15','t', msl_200_vo, t_q), 'coolwarm',250,290)

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','04','02','15','vo', msl_200_vo, idk), 'viridis', -0.0001, 0.0005)

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','04','02','15','vo', msl_200_vo, idk2), 'viridis', -0.0002, 0.0004)

In [ ]:
tp_cp = xr.open_dataset('/home/glach/projects/def-rfajber/shared/gabe-fall-2025/TEtracking/data/era5-raw/2015-tp-cp.nc')

In [ ]:
base_cmap = plt.get_cmap('gist_ncar')
cmap_colors = base_cmap(np.arange(base_cmap.N))
cmap_colors[0, -1] = 0  
transparent_cmap = mcolors.ListedColormap(cmap_colors)

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','04','02','15','tp', msl_200_vo, tp_cp), transparent_cmap)

##########################

In [ ]:
## anticyclone

plotMSLAtTime(findDataAtTime('2015','03','14','12','msl', anticyclone_150, msl_u_v))

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','03','14','12','t', anticyclone_150, t_q), 'coolwarm',250,290)

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','03','14','12','vo', anticyclone_150, idk), 'viridis', -0.0001, 0.0005)

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','03','14','12','vo', anticyclone_150, idk2), 'viridis', -0.0002, 0.0004)

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','03','14','12','tp', anticyclone_150, tp_cp), transparent_cmap)

In [ ]:
outp1 = (findDataAtTime('2015','04','02','15','msl', msl_200_vo, msl_u_v))

In [ ]:
outp2 = (findDataAtTime('2015','04','02','15','t', msl_200_vo, t_q))

In [ ]:
a = plotMSLAndGradAtTime(outp1, outp2,'coolwarm',250,290)

In [ ]:
outp1 = (findDataAtTime('2015','04','02','15','msl', msl_200_vo, msl_u_v))

In [ ]:
outp2 = findDataAtTime('2015','04','02','15','vo', msl_200_vo, idk)

In [ ]:
plotMSLAndGradAtTime(outp1, outp2,'viridis', -0.0001, 0.0005)

In [ ]:
## cyclone, t 850

plotGRADAtTime(findDataAtTime('2015','03','14','12','t', anticyclone_150, t_q), 'coolwarm',250,290)

In [ ]:
outp1_2 = findDataAtTime('2015','03','14','12','msl', anticyclone_150, msl_u_v)

In [ ]:
outp2_2 = findDataAtTime('2015','03','14','12','t', anticyclone_150, t_q)

In [ ]:
plotMSLAndGradAtTime(outp1_2, outp2_2,'coolwarm',250,290)

In [ ]:
outp1_2 = findDataAtTime('2015','03','14','12','msl', anticyclone_150, msl_u_v)

In [ ]:
outp2_2 = findDataAtTime('2015','03','14','12','vo', anticyclone_150, idk)

In [ ]:
plotMSLAndGradAtTime(outp1_2, outp2_2,'viridis', -0.0001, 0.0005)